In [151]:
# 필요한 라이브러리들을 로드하자
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

In [152]:
# 인베스팅닷컴 열기
driver = uc.Chrome()
url = 'https://www.marketvector.com/indices/customized/atac-risk-on-risk-off-domestic'  # 원하는 웹사이트 주소로 변경하세요
driver.get(url)

actions = ActionChains(driver)

In [153]:
element = driver.find_elements(By.CSS_SELECTOR, '.chartCheckBoxText')
actions.click(element[1]).perform() 

In [167]:
element2 = driver.find_element(By.CSS_SELECTOR, '[data-changerange="5Y"]')
actions.click(element2).perform()

In [207]:
# 내가 수동으로 찾은 초기 커서 위치로서 적합한 커서 위치
element3 = driver.find_elements(By.CSS_SELECTOR, '.chartLegendBox')

# 찾은 요소를 클릭합니다.
actions.click(element3[1]).perform()   #actions.click(element3[1]).perform()   # 우클릭 하고 싶으면 actions.context_click(element3).perform()

In [208]:
# 첫 데이터가 나타나는 곳으로 커서를 이동후 클릭(내가 수동으로 찾음)
actions.move_by_offset(-11, 180).perform()

In [209]:
data_list = []

for i in range(729):

    # 페이지의 HTML을 가져옴
    html = driver.page_source
    # BeautifulSoup 객체를 생성
    soup = bs(html, 'html.parser')
    # 클래스가 "chart-wrapper"인 요소를 찾아서 해당하는 것을 data에 저장
    wrapper = soup.find('g', attrs={'class': 'highcharts-label highcharts-tooltip-box highcharts-color-none'})
    wrapper2 = soup.find('g', attrs={'class': 'highcharts-label highcharts-tooltip-header highcharts-tooltip-box highcharts-color-none'})
    data = wrapper.prettify()
    data2 = wrapper2.prettify()

    # 데이터에서 요일, 월, 일, 연도 추출
    date_start_index = data2.find('<tspan style="font-size: 10px">\n') + len('<tspan style="font-size: 10px">\n')
    date_end_index = data2.find("</tspan>")
    date = data2[date_start_index:date_end_index].strip()
    # 날짜 잘 나오나 함 찍어보자
    print(date)

    # 데이터에서 RoRo 값을 추출
    number_start_index = data.find('<tspan style="font-weight:bold;">') + len('<tspan style="font-weight:bold;">')
    number_end_index = data.find("</tspan>", number_start_index)
    number = data[number_start_index:number_end_index].strip()
    # ESI값 잘 나오나 함 찍어보자
    print(number)

    # 추출한 정보를 리스트에 형태로 하나씩 추가
    data_list.append({'date': date, 'number': number})

    # 커서를 한 칸씩 옮겨보자
    actions.move_by_offset(1, 0).click().perform()

# 커서를 하나씩 이동 시켜서 하나씩 크롤링 하는 작업을 끝냈으면 해당 데이터를 데이터 프레임으로 저장
df = pd.DataFrame(data_list)

Tuesday, Jul  3, 2018
0.00%
Friday, Jul  6, 2018
2.64%
Monday, Jul  9, 2018
3.70%
Wednesday, Jul 11, 2018
2.67%
Friday, Jul 13, 2018
3.79%
Tuesday, Jul 17, 2018
4.06%
Wednesday, Jul 18, 2018
4.31%
Friday, Jul 20, 2018
4.27%
Tuesday, Jul 24, 2018
3.33%
Thursday, Jul 26, 2018
2.97%
Friday, Jul 27, 2018
3.14%
Tuesday, Jul 31, 2018
3.35%
Thursday, Aug  2, 2018
2.43%
Friday, Aug  3, 2018
2.94%
Tuesday, Aug  7, 2018
2.46%
Thursday, Aug  9, 2018
3.43%
Monday, Aug 13, 2018
3.96%
Tuesday, Aug 14, 2018
3.73%
Thursday, Aug 16, 2018
4.33%
Monday, Aug 20, 2018
4.80%
Tuesday, Aug 21, 2018
5.73%
Thursday, Aug 23, 2018
5.82%
Monday, Aug 27, 2018
7.44%
Wednesday, Aug 29, 2018
8.40%
Thursday, Aug 30, 2018
8.12%
Tuesday, Sep  4, 2018
8.16%
Thursday, Sep  6, 2018
6.59%
Monday, Sep 10, 2018
6.81%
Tuesday, Sep 11, 2018
6.00%
Thursday, Sep 13, 2018
6.41%
Monday, Sep 17, 2018
5.90%
Tuesday, Sep 18, 2018
4.79%
Thursday, Sep 20, 2018
4.65%
Monday, Sep 24, 2018
4.33%
Wednesday, Sep 26, 2018
4.96%
Thursday, Sep 2

In [219]:
df

,date,number
0,"Tuesday, Jul 3, 2018",0.00%
1,"Friday, Jul 6, 2018",2.64%
2,"Monday, Jul 9, 2018",3.70%
3,"Wednesday, Jul 11, 2018",2.67%
4,"Friday, Jul 13, 2018",3.79%
...,...,...
724,"Tuesday, Jun 20, 2023",48.03%
725,"Thursday, Jun 22, 2023",47.06%
726,"Monday, Jun 26, 2023",43.83%
727,"Wednesday, Jun 28, 2023",47.27%


In [220]:
data1 = df.copy()
data1.set_index('date', inplace=True)
data1.index = pd.to_datetime(data1.index)
data1.index = pd.to_datetime(data1.index.date)

In [221]:
data1.to_csv('RORO.csv')

In [224]:
pd.read_csv('RORO.csv')

,Unnamed: 0,number
0,2018-07-03,0.00%
1,2018-07-06,2.64%
2,2018-07-09,3.70%
3,2018-07-11,2.67%
4,2018-07-13,3.79%
...,...,...
724,2023-06-20,48.03%
725,2023-06-22,47.06%
726,2023-06-26,43.83%
727,2023-06-28,47.27%
